In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install uv
!uv pip install trimesh
!uv pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 110.8 MB/s eta 0:00:00
Using Python 3.11.11 environment at: /usr
Resolved 2 packages in 133ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
Prepared 1 package in 52ms
Installed 1 package in 7ms
 + trimesh==4.6.4
Using Python 3.11.11 environment at: /usr
Resolved 3 packages in 98ms
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
lz4        ------------------------------     0 B/1.21 MiB
⠙ Preparing packages... (0/2)
lz4        -------

Importar libs

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Lambda
from tensorflow.keras.models import Sequential
import cv2
import os
from sklearn.metrics.pairwise import cosine_similarity
import gdown
import trimesh
import zipfile
import random
from tqdm import tqdm

In [4]:
# Definir seed para reprodutibilidade
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
# random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Configurações adicionais para garantir reprodutibilidade no TensorFlow
tf.keras.utils.set_random_seed(seed)  # Configura seed para keras, tensorflow e numpy
tf.config.experimental.enable_op_determinism()  # Garante operações determinísticas

Extração dos dados

In [5]:
# Baixar os dados de treino
output_path = '/content/kitti_dataset.zip'
url = f'https://drive.google.com/uc?id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT'

gdown.download(url, output_path, quiet=False)

# Extrair o conteúdo
extract_dir = output_path.replace(".zip", "")
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Ajuste aqui: usar a pasta "post-processed" dentro do diretório extraído
post_processed_dir = os.path.join(extract_dir, "post-processed")

# Verificar se a pasta existe
if not os.path.exists(post_processed_dir):
    print(f"Pasta 'post-processed' não encontrada em {extract_dir}")
    print(f"Conteúdo do diretório: {os.listdir(extract_dir)}")
else:
    print(f"Pasta 'post-processed' encontrada! Conteúdo: {os.listdir(post_processed_dir)}")


Downloading...
From (original): https://drive.google.com/uc?id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT
From (redirected): https://drive.google.com/uc?id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT&confirm=t&uuid=818aeb7d-6829-4bab-8048-b1c1bfab0ad8
To: /content/kitti_dataset.zip
100%|██████████| 71.4M/71.4M [00:01<00:00, 64.5MB/s]


Pasta 'post-processed' encontrada! Conteúdo: ['Samira_Makhmalbaf', 'Woody_Allen', 'David_Caruso', 'John_Wright', 'Mark_Kelly', 'Thad_Matta', 'Jane_Kaczmarek', 'Laurel_Clark', 'Leonardo_DiCaprio', 'Roy_Williams', 'Peter_Mugyeni', 'Jennifer_Reilly', 'Raul_Rivero', 'Linda_Dano', 'Ben_Braun', 'Keira_Knightley', 'Jim_Bunning', 'Miranda_Otto', 'Alan_Ball', 'Asmaa_Assad', 'Robin_Tunney', 'Lisa_Murkowski', 'Qusai_Hussein', 'Dionne_Warwick', 'Steve_Austin', 'Fran_Drescher', 'Priscilla_Presley', 'Jean-Claude_Juncker', 'Imelda_Marcos', 'Marc-Andre_Fleury', 'Jennifer_Garner', 'Paul_Bremer', 'Oswaldo_Paya', 'Alejandro_Lerner', 'Laurent_Woulzy', 'Raymond_Odierno', 'Yasein_Taher', 'Tom_Cruise', 'Ivan_Stambolic', 'Jorge_Arce', 'Jonathan_Mostow', 'Shigeru_Ishiba', 'Rick_Carlisle', 'Bill_Clinton', 'Dwayne_Wade', 'Jean-Marc_de_La_Sabliere', 'Mike_Scioscia', 'Stuart_Townsend', 'Bill_Cartwright', 'Bob_Hayes', 'George_McCloud', 'Javier_Vazquez', 'Adolfo_Rodriguez_Saa', 'Gloria_Allred', 'Pete_Rose', 'Carlos_

In [6]:
folders = sorted(os.listdir(post_processed_dir))

original_img_dataset = pd.DataFrame(columns=['img', 'id'])

for idx, folder in enumerate(folders):
    folder_path = os.path.join(post_processed_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                img_path = os.path.join(folder_path, filename)
                img_original = cv2.imread(img_path)
                img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
                try:
                    # Dataset de imagens originais (RGB)
                    original_img_dataset = pd.concat([original_img_dataset, pd.DataFrame({'img': [img_original], 'id': [idx]})], ignore_index=True)
                except Exception as e:
                    print(f"Erro ao carregar {img_path}: {e}")

print('Número total de imagens carregadas:', len(original_img_dataset))

Número total de imagens carregadas: 12000


In [7]:
# def preprocess_image(input_img):
#     # Verificar se a entrada é um caminho ou uma imagem
#     img = input_img.copy()

#     detector = MTCNN(device="GPU:0")
#     faces = detector.detect_faces(img)
#     if len(faces) > 0:
#         # Extrair a região do rosto com margens extras
#         x, y, width, height = faces[0]['box']
#         # Expandir um pouco a caixa para incluir toda a testa
#         y_expanded = max(0, y - int(height * 0.2))
#         height_expanded = int(height * 1.2)
#         face_img = img[y_expanded:y_expanded+height_expanded, x:x+width]
#     else:
#         # Fallback quando nenhum rosto é detectado - remover a imagem
#         face_img = None

#     return face_img

In [10]:
def preprocess_image(input_img):
    # Verificar se a entrada é um caminho ou uma imagem
    img = input_img.copy()

    detector = MTCNN(device="GPU:0")  # You can change this to "CPU" if needed
    faces = detector.detect_faces(img)

    if len(faces) > 0:
        # Extrair a região do rosto com margens extras
        x, y, width, height = faces[0]['box']
        # Expandir um pouco a caixa para incluir toda a testa
        y_expanded = max(0, y - int(height * 0.2))
        height_expanded = int(height * 1.2)
        face_img = img[y_expanded:y_expanded+height_expanded, x:x+width]
    else:
        # Fallback quando nenhum rosto é detectado - usar a imagem original
        # ou aplicar outro processamento alternativo
        face_img = None

    return face_img


Preparar dados de treinamento

In [ ]:
img_dataset = pd.DataFrame(columns=['img', 'id'])

# Usar tqdm para mostrar a barra de progresso
for idx, row in tqdm(original_img_dataset.iterrows(), total=len(original_img_dataset), desc="Processando imagens"):
    try:
        # Processar a imagem usando preprocess_image
        processed_img = preprocess_image(row['img'])

        # Adicionar ao dataset de imagens processadas
        img_dataset = pd.concat([img_dataset, pd.DataFrame({'img': [processed_img], 'id': [row['id']]})], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar imagem com ID {row['id']}: {e}")

print('Número total de imagens processadas:', len(img_dataset))

Processando imagens:   8%|▊         | 933/12000 [04:03<47:18,  3.90it/s]

In [ ]:
# print first 5 images

for i in range(5):
    plt.figure(figsize=(2, 2))
    plt.imshow(original_img_dataset['img'][i])
    plt.axis('off')
    plt.show()

In [ ]:
# print first 5 images

for i in range(5):
    plt.figure(figsize=(2, 2))
    plt.imshow(img_dataset['img'][i])
    plt.axis('off')
    plt.show()

In [ ]:
X = np.stack(img_dataset['img'].values)
y = img_dataset['id'].values

Configurar modelo

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Construir o modelo sequencial completo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(128),
    Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))
])

### Âncora
- É a imagem de referência (ex: uma foto da pessoa A)
- Serve como ponto de comparação para as outras duas imagens

### Positivo
- É outra imagem da **mesma pessoa** da âncora (outra foto da pessoa A)
- Pode ter diferenças como ângulo, iluminação, expressão facial, uso de máscara

### Negativo
- É uma imagem de uma **pessoa diferente** (foto da pessoa B)
- Representa o que o modelo deve aprender a distinguir da âncora

In [ ]:
# A classe TripletGenerator é um gerador de dados para o treinamento do modelo
class TripletGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size=32, num_triplets=5000, **kwargs):
        super().__init__(**kwargs)
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_triplets = num_triplets

        # Encontrar classes com pelo menos 2 amostras
        unique_labels, counts = np.unique(labels, return_counts=True)
        self.classes = unique_labels[counts >= 2]

        if len(self.classes) == 0:
            raise ValueError("Não há classes com pelo menos 2 amostras. Impossível criar triplets.")

        self.indexes = np.arange(num_triplets)


    def __len__(self):
        return int(np.ceil(self.num_triplets / self.batch_size))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_triplets = []

        for _ in range(len(batch_indexes)):
            # Escolher classe âncora aleatoriamente (apenas entre classes com múltiplas amostras)
            anchor_class = np.random.choice(self.classes)

            # Índices de imagens da mesma classe (positivos)
            positive_indices = np.where(self.labels == anchor_class)[0]

            # Verificar se ainda temos pelo menos 2 imagens nesta classe
            if len(positive_indices) < 2:
                continue

            # Escolher âncora e positivo aleatoriamente da mesma classe
            anchor_idx = np.random.choice(positive_indices)
            positive_candidates = positive_indices[positive_indices != anchor_idx]

            # Verificação extra para garantir que temos positivos após remover o âncora
            if len(positive_candidates) == 0:
                continue

            positive_idx = np.random.choice(positive_candidates)

            # Escolher negativo de outra classe
            negative_classes = self.classes[self.classes != anchor_class]
            # Verificar se temos outras classes disponíveis
            if len(negative_classes) == 0:
                continue

            negative_class = np.random.choice(negative_classes)
            negative_indices = np.where(self.labels == negative_class)[0]
            negative_idx = np.random.choice(negative_indices)

            batch_triplets.append([
                self.images[anchor_idx],
                self.images[positive_idx],
                self.images[negative_idx]
            ])

        # Garantir que temos pelo menos um triplet no batch
        if not batch_triplets:
            # Tenta novamente com outro índice
            return self.__getitem__((idx + 1) % len(self))

        batch_triplets = np.array(batch_triplets)
        triplets_flat = batch_triplets.reshape(-1, 224, 224, 3)
        targets = np.zeros(len(batch_triplets) * 3)

        return triplets_flat, targets

In [ ]:
# A loss function triplet_loss calcula a perda triplet, que é uma função que mede a distância entre âncoras, positivos e negativos
# A ideia é minimizar a distância entre âncoras e positivos, enquanto maximiza a distância entre âncoras e negativos
def triplet_loss(y_true, y_pred, alpha=0.2):
	total = tf.shape(y_pred)[0]
	third = total // 3

	anchor = y_pred[0:third]
	positive = y_pred[third:2*third]
	negative = y_pred[2*third:total]

	# Calcular distâncias entre âncora, positivo e negativo
	pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
	neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)

	# Triplet Loss
	basic_loss = pos_dist - neg_dist + alpha
	loss = tf.maximum(basic_loss, 0.0)
	return tf.reduce_mean(loss)

Compilar e treinar o modelo

In [ ]:
# Compilar o modelo com a função de perda triplet loss
model.compile(optimizer='adam', loss=triplet_loss)

# Cria o gerador de triplets
triplet_gen = TripletGenerator(X, y, batch_size=32, num_triplets=5000)

# Treina o modelo com o gerador
history = model.fit(
    triplet_gen,
    epochs=10,
    steps_per_epoch=len(triplet_gen),
    verbose=1
)

Criar banco de dados com vetores descritores

In [ ]:
# Função para extrair vetores descritores
def extract_features(model, images):
    features = model.predict(images)
    return features

In [ ]:
# Extrair características de todas as imagens do dataset
celebrity_features = extract_features(model, X)

# Criar banco de dados de características
feature_database = {
    'features': celebrity_features,
    'labels': y
}

# # Podemos também salvar para uso futuro
# np.save('feature_database.npy', feature_database)

Adicionar nova pessoa ao banco

In [ ]:
# Carregar a imagem da nova pessoa
new_person_path = '/content/marcelinho_no_db.jpg'

url = "https://drive.google.com/uc?id=1EgvzTNEWTXvegURlmJAt8OXOtrKAlQEb"
gdown.download(url, new_person_path, quiet=False)

# Carregar a imagem
img_original = cv2.imread(new_person_path)
img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

# Processar a imagem usando preprocess_image com a imagem já carregada
new_person_img = preprocess_image(img_original)
new_person_img_array = np.expand_dims(new_person_img, axis=0)

# Extrair características
new_person_features = extract_features(model, new_person_img_array)

# Atribuir um novo ID para esta pessoa (após o último ID existente)
new_person_id = np.max(y) + 1

# Adicionar ao banco de dados
original_img_dataset = pd.concat([original_img_dataset, pd.DataFrame({'img': [img_original], 'id': [new_person_id]})], ignore_index=True)
feature_database['features'] = np.vstack([feature_database['features'], new_person_features])
feature_database['labels'] = np.append(feature_database['labels'], new_person_id)

print(f"Nova pessoa adicionada ao banco de dados com ID {new_person_id}")


Teste final

In [ ]:
test_image_path = "face-test\marcelinho_na_inferencia.jpg"

# Adicionar ao banco de dados com imagens originais
img_original = cv2.imread(test_image_path)
img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
original_img_dataset = pd.concat([original_img_dataset, pd.DataFrame({'img': [img_original], 'id': [new_person_id]})], ignore_index=True)

plt.figure(figsize=(6, 6))
plt.imshow(img_original)
plt.title("Imagem original de pessoa a ser buscada")
plt.axis('off')
plt.show()

# Imagem não é adicionada ao banco de dados, apenas pré-processada e utilizada na inferência
test_image = preprocess_image(test_image_path)
test_image_array = np.expand_dims(test_image, axis=0)

# Exibir a imagem a ser buscada
plt.figure(figsize=(6, 6))
plt.imshow(test_image)
plt.title("Imagem processada de pessoa a ser buscada")
plt.axis('off')
plt.show()

In [ ]:
test_image_path = "/content/marcelinho_na_inferencia.jpg"
url = "https://drive.google.com/uc?id=1RcLasSJj-XMke5Fj33adiaHWbbl-9ihW"
gdown.download(url, test_image_path, quiet=False)

# Carregar a imagem
img_original = cv2.imread(test_image_path)
img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

# Adicionar ao banco de dados com imagens originais
original_img_dataset = pd.concat([original_img_dataset, pd.DataFrame({'img': [img_original], 'id': [new_person_id]})], ignore_index=True)

plt.figure(figsize=(6, 6))
plt.imshow(img_original)
plt.title("Imagem original de pessoa a ser buscada")
plt.axis('off')
plt.show()

# Usar a imagem já carregada para o pré-processamento, em vez do caminho do arquivo
test_image = preprocess_image(img_original)
test_image_array = np.expand_dims(test_image, axis=0)

# Exibir a imagem a ser buscada
plt.figure(figsize=(6, 6))
plt.imshow(test_image)
plt.title("Imagem processada de pessoa a ser buscada")
plt.axis('off')
plt.show()
